#@title 🚀 蒼狼 AI 分析平台：一鍵啟動中心
#@markdown ---
#@markdown ### **歡迎使用！只需兩步，即可啟動您的專屬 AI 分析助理。**
#@markdown **👇 請先完成以下設定，然後點擊左側的「播放」按鈕執行。**
#@markdown 
#@markdown --- 
#@markdown ### 步驟 1: 選擇您的工作模式 
#@markdown **操作模式 (`operation_mode`)**: 決定您的資料要存放在哪裡。
#@markdown - **嚐鮮一下 (transient)**：<font color='green'>**推薦新用戶！**</font> 無需授權，啟動最快。所有資料（報告、對話）在連線中斷後會消失，適合快速體驗。
#@markdown - **正式使用 (persistent)**：<font color='blue'>**推薦長期使用！**</font> 需要授權 Google Drive。您的資料將被安全地保存在您的雲端硬碟，可隨時回來繼續工作。
operation_mode = "transient" #@param ["transient", "persistent"]

#@markdown <br>

#@markdown --- 
#@markdown ### 步驟 2: 選擇技術選項 (可保持預設)
#@markdown **程式碼分支 (`branch_name`)**: 一般使用者請保持 `main` 即可。
branch_name = "main" #@param {type:"string"}
#@markdown **啟動模式 (`MODE`)**: 若您不是開發者，請保持 `normal` 即可。
MODE = "normal" #@param ["normal", "debug"]
#@markdown 
#@markdown --- 
#@markdown ✅ 設定完成！請點擊左側的 **▶️ (播放) 按鈕** 開始部署，過程約需 2-3 分鐘。

In [ ]:
import os
import shutil
import time
import requests
from IPython.display import display, HTML, clear_output
from datetime import datetime
import pytz # 引入時區函式庫

# 清理輸出區域，提供更清爽的日誌體驗
clear_output(wait=True)
print(f"您選擇的操作模式是：{operation_mode}")
print(f"您選擇的啟動模式是：{MODE}")

# 設定環境變數
os.environ['OPERATION_MODE'] = operation_mode
print(f"環境變數 OPERATION_MODE 已設定為: {os.getenv('OPERATION_MODE')}")

print("INFO: 開始執行 Colab 啟動程序...")
TAIPEI_TZ = pytz.timezone('Asia/Taipei') # 定義台北時區

def get_taipei_time_str():
    """返回格式化的台北時間字串"""
    return datetime.now(TAIPEI_TZ).strftime('%Y-%m-%d %H:%M:%S %Z')

print(f"\n[{get_taipei_time_str()}] STEP 1: 檢查是否需要掛載 Google Drive...")
if operation_mode == "persistent":
    print(f"[{get_taipei_time_str()}] INFO: 持久模式 - 開始掛載 Google Drive...")
    try:
        from google.colab import drive
        drive.mount('/content/drive', force_remount=True)
        print(f"[{get_taipei_time_str()}] SUCCESS: Google Drive 已成功掛載到 /content/drive")
    except Exception as e:
        print(f"[{get_taipei_time_str()}] ERROR: 掛載 Google Drive 失敗: {e}")
        raise SystemExit("Google Drive 掛載失敗，請處理後重試。")
else:
    print(f"[{get_taipei_time_str()}] INFO: 暫存模式 - 跳過 Google Drive 掛載。")

print(f"\n[{get_taipei_time_str()}] STEP 2 & 3: Colab Secrets 設定引導與檢查...")
all_secrets_ok = True
if not os.getenv('COLAB_GOOGLE_API_KEY'):
    print(f"[{get_taipei_time_str()}]   ⚠️ COLAB_GOOGLE_API_KEY 未設定。AI 分析功能可能無法使用。")
else:
    print(f"[{get_taipei_time_str()}]   ✅ COLAB_GOOGLE_API_KEY 已設定。")

if operation_mode == "persistent":
    print(f"[{get_taipei_time_str()}] INFO: 持久模式 - 檢查 Google Drive 相關 Colab Secrets...")
    secrets_to_check_persistent = ['GOOGLE_SERVICE_ACCOUNT_JSON_CONTENT', 'WOLF_IN_FOLDER_ID', 'WOLF_PROCESSED_FOLDER_ID']
    missing_secrets_persistent = [s for s in secrets_to_check_persistent if not os.getenv(s)]
    if missing_secrets_persistent:
        all_secrets_ok = False
        for secret_name in missing_secrets_persistent:
            print(f"[{get_taipei_time_str()}]   ❌ {secret_name} 未設定或為空值！")
    else:
        print(f"[{get_taipei_time_str()}]   ✅ 持久模式所需的所有 Secrets 均已設定。")
else:
    print(f"\n[{get_taipei_time_str()}] INFO: 暫存模式 - 跳過 Google Drive 相關 Colab Secrets 的檢查。")

if not all_secrets_ok:
    raise SystemExit("❌ 錯誤：持久模式下，必要的 Colab Secrets 設定不完整，無法繼續。")
else:
    print(f"\n[{get_taipei_time_str()}] ✅ Colab Secrets 設定檢查完畢。")

print(f"\n[{get_taipei_time_str()}] STEP 4: 複製專案程式碼...")
GIT_REPO_URL = "https://github.com/hsp1234-web/wolfAI_v1.git"
PROJECT_PARENT_DIR = "/content/wolf_project"
PROJECT_DIR_NAME = "wolfAI_v1"
FULL_PROJECT_PATH = os.path.join(PROJECT_PARENT_DIR, PROJECT_DIR_NAME)

if os.path.exists(FULL_PROJECT_PATH):
    shutil.rmtree(FULL_PROJECT_PATH)
os.makedirs(PROJECT_PARENT_DIR, exist_ok=True)
clone_command = f"git clone --depth 1 -b {branch_name} {GIT_REPO_URL} {FULL_PROJECT_PATH}"
clone_status = os.system(clone_command)

if clone_status != 0 or not os.path.isdir(FULL_PROJECT_PATH):
    raise SystemExit(f"ERROR: 專案複製失敗。請檢查 URL ('{GIT_REPO_URL}') 和分支 ('{branch_name}')。")
else:
    print(f"[{get_taipei_time_str()}] SUCCESS: 專案已成功複製到 '{FULL_PROJECT_PATH}'。")

print(f"\n[{get_taipei_time_str()}] STEP 5: 執行主控啟動腳本 scripts/start.sh...")
start_script_path = os.path.join(FULL_PROJECT_PATH, 'scripts', 'start.sh')
if os.path.exists(start_script_path):
    os.system(f"chmod +x {start_script_path}")
    start_script_command = f"{start_script_path} --mode={MODE}"
    exit_code = os.system(start_script_command)
    if exit_code != 0:
        print(f"[{get_taipei_time_str()}] ERROR: 啟動腳本執行失敗，返回碼: {exit_code}。請檢查日誌。")
else:
    raise SystemExit(f"ERROR: 主控啟動腳本 {start_script_path} 未找到！")

print(f"\n[{get_taipei_time_str()}] STEP 6: 健康度偵測 (Health Check) - 請耐心等待約 1-2 分鐘...")
BACKEND_SERVICE_PORT = 8000
FRONTEND_SERVICE_PORT = 3000
max_wait_duration_seconds = 120
check_interval_seconds = 10
start_check_time = time.time()
backend_service_ready, frontend_service_ready = False, False

while time.time() - start_check_time < max_wait_duration_seconds:
    # 在迴圈的每一次迭代都獲取最新的台北時間
    current_time_str = get_taipei_time_str()
    try:
        if not backend_service_ready and requests.get(f"http://localhost:{BACKEND_SERVICE_PORT}/api/health", timeout=5).status_code == 200:
            backend_service_ready = True
            print(f"[{current_time_str}] ✅ 後端服務 READY.")
    except requests.RequestException:
        print(f"[{current_time_str}] ⏳ 後端服務未就緒...")

    try:
        if not frontend_service_ready and requests.get(f"http://localhost:{FRONTEND_SERVICE_PORT}", timeout=5).status_code == 200:
            frontend_service_ready = True
            print(f"[{current_time_str}] ✅ 前端服務 READY.")
    except requests.RequestException:
        print(f"[{current_time_str}] ⏳ 前端服務未就緒...")

    if backend_service_ready and frontend_service_ready:
        print(f"\n🎉 太棒了！後端和前端服務均已成功啟動！ ({get_taipei_time_str()})")
        break

    if time.time() - start_check_time >= max_wait_duration_seconds:
        print(f"\n⏱️ 服務啟動或健康檢查超時。 ({get_taipei_time_str()})")
        break

    time.sleep(check_interval_seconds)

print(f"\n[{get_taipei_time_str()}] STEP 7: 顯示結果...")
if backend_service_ready and frontend_service_ready:
    border = "="*70
    print(f"\n{border}")
    display(HTML("<h2>✅ 應用程式已就緒，可以開始使用了！</h2>"))
    print("🔗 前端應用程式訪問網址 (Colab Proxy URL):")
    print(f"   請在本儲存格的輸出區域上方尋找由 Colab 自動生成的、形如 https://*.googleusercontent.com/ 的公開連結。")
    print(f"{border}\n")
else:
    print("\n❌ 很抱歉，一個或多個服務未能成功啟動。")
    print("   請仔細檢查上方的日誌輸出以了解詳細錯誤信息。")

print(f"INFO: Colab 啟動程序執行完畢。 ({get_taipei_time_str()})")